## Real-time epidemic modelling of SARS-CoV-2 from wastewater data

This is a sandbox for EDA and mathematical modelling of R estimate from viral RNA level in wastewater.

Equation from original paper: <br>

&emsp; $P = \frac{C \times P}{S \times Q} $

$P$: Prevalence <br>
$C$: Viral RNA concentration in wastewater <br>
$S$: Shedding rate per day <br>
$Q$: Volume of wastewater produced by a person per day <br>

Equation from Achaiah et al. 2020:

&emsp; $ I_{t} = \frac{(R_{0})t}{SI} $

$I_{t}$ : number of cases at the time <br>
$R_{0}$: reproduction number <br>
$t$ : prediction time <br>
$SI$ : serial interval <br>

In [177]:
! pip install Plotly
! pip install cufflinks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.7/81.7 KB 1.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for cufflinks: filename=cufflinks-0.17.3-py3-none-any.whl size=67918 sha256=de15092ac7d38d5c45e635c0d657beba4b71964a76637ea07793b723f64abca9
  Stored in directory: /Users/fm/Library/Caches/pip/wheels/09/8a/6b/cbe3e87b2e59bb5f90b49b034ce36b80b46a4d6e38444c34de
Successfully built cufflinks


In [178]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
import cufflinks as cf
plt.style.use('seaborn-white')

%matplotlib inline

from scipy.stats import gamma, poisson

import epyestim
import epyestim.covid19 as covid19
import subprocess # for calling R scripts

# read in and preview data
df = pd.read_csv('RNAMonitoring.txt', encoding = 'utf16', delimiter = "\t")
df.head()

,SiteCode,Date,Population Band,Population,Result Description - N1 Gene,Reported Value - N1 Gene (gc/l),Days Since Sampled
0,ALLA,01/06/2020,10k - 100k,62058,Negative,0,703 days
1,ALLA,02/06/2020,10k - 100k,62058,Negative,0,702 days
2,ALLA,04/06/2020,10k - 100k,62058,Negative,0,700 days
3,ALLA,08/06/2020,10k - 100k,62058,Negative,0,696 days
4,ALLA,09/06/2020,10k - 100k,62058,Negative,0,695 days


In [179]:
# Show column names
# list(df)

# Rename columns
df.columns = ['sitecode', 'date', 'popband', 'pop', 'result_desc', 'gene', 'days_sampled']

# Inspect data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10518 entries, 0 to 10517
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   sitecode      10518 non-null  object
 1   date          10518 non-null  object
 2   popband       10518 non-null  object
 3   pop           10518 non-null  object
 4   result_desc   10518 non-null  object
 5   gene          10518 non-null  int64 
 6   days_sampled  10518 non-null  object
dtypes: int64(1), object(6)
memory usage: 575.3+ KB


In [180]:
# Convert datatypes and set factors
df['date'] = pd.to_datetime(df['date'], dayfirst=True)
df['popband'] = df['popband'].astype('category')


In [181]:
# Check updated data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10518 entries, 0 to 10517
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   sitecode      10518 non-null  object        
 1   date          10518 non-null  datetime64[ns]
 2   popband       10518 non-null  category      
 3   pop           10518 non-null  object        
 4   result_desc   10518 non-null  object        
 5   gene          10518 non-null  int64         
 6   days_sampled  10518 non-null  object        
dtypes: category(1), datetime64[ns](1), int64(1), object(4)
memory usage: 503.6+ KB


In [182]:
# Check number of unique sites
df.sitecode.nunique() #121 sites

121

In [183]:
# Frequency of each site
print(df.sitecode.value_counts())

SHIE    290
ALLA    280
KIRK    268
DALM    252
BLAC    240
       ... 
BARR      1
GOUR      1
LARK      1
SERL      1
INVU      1
Name: sitecode, Length: 121, dtype: int64


In [184]:
site_groups = df.groupby(df['sitecode'])
site_groups.mean()

,gene
sitecode,
ALLA,95745.153571
ALLE,218549.716981
ALLO,106434.160305
ALNE,264837.250000
ALVA,157006.258427
...,...
TURR,114550.032258
UPLA,390.307692
WHIT,155456.417476
